In [1]:
import pandas as pd
import plotly.express as px
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
def get_champion_tierlist(patch:str = '13.9'):
    driver = webdriver.Chrome()
    url = f"https://lolalytics.com/lol/tierlist/?patch={patch}"

    driver.get(url)
    wait = WebDriverWait(driver, 60)
    local_class = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='root']/div[6]/div/div")))
    element_class = local_class.get_attribute("class")

    soup = BeautifulSoup(driver.page_source)

    tier_list = {}
    tierlist_div = soup.find('div', attrs={'class': element_class}).find_all('div', recursive=False)

    for row in tierlist_div:
        columns = row.find_all('div')

        tier_list[columns[1].text] = {
            "tier":columns[2].text,
            "pickLane":columns[3].text,
            "winRate":columns[6].text,
            "delta":columns[7].text,
            "pickRate":columns[8].text,
            "banRate":columns[9].text,
            "games":columns[10].text,
        }

    driver.close()
    return tier_list

In [3]:
def get_tierlist(patch_list: list):

    data_frames_aggregated = {}
    for patch in patch_list:
        data = pd.DataFrame(get_champion_tierlist(patch)).T
        data_frames_aggregated[patch] = data

    data = pd.concat(data_frames_aggregated).reset_index()
    data2 =  data.sort_values(by=['level_0', 'level_1'])

    numeric_data = data2.copy()
    cond = numeric_data[numeric_data.level_0 == numeric_data.level_0.min()]['level_1']

    columns = ['level_0','level_1']
    columns.extend(list(data2.columns[3:]))

    numeric_data_filtered = data2[data2['level_1'].isin(cond)]
    numeric_data_filtered = numeric_data_filtered[columns].set_index('level_0')

    tierlist_values = numeric_data_filtered.iloc[:,1:-1].astype('Float64')
    tierlist_games = numeric_data_filtered.iloc[:,-1].str.replace('.', '')
    tierlist_games = tierlist_games.astype('Float64')
    tierlist_champions = numeric_data_filtered['level_1']

    tierlist_dataframe = pd.concat(
        [
            tierlist_champions,
            tierlist_values,
            tierlist_games
        ],
        axis=1,
        )

    tierlist_dataframe.rename(columns={"level_1":"champion"},inplace=True)
    return tierlist_dataframe

In [4]:
def fig_tierlist(tierlist_df: pd.DataFrame, information: str = "winRate"):

    fig = px.line(
        tierlist_df,
        x="champion",
        y=information,
        color=tierlist_df.index
    )

    fig.update_layout(
        title=f"{information} between patches",
        xaxis_title="Champion",
        yaxis_title=f"{information}"
    )

    return fig

In [5]:
def fig_tierlist_grouped(tierlist_df: pd.DataFrame, information: str = "winRate", method: str = "mean"):

    if method == "mean":
        grouped_champ_df = tierlist_df.groupby(by=['champion']).mean()
    elif method == "median":
        grouped_champ_df = tierlist_df.groupby(by=['champion']).median()
    elif method == "std":
        grouped_champ_df = tierlist_df.groupby(by=['champion']).std()
    elif method == "max":
        grouped_champ_df = tierlist_df.groupby(by=['champion']).max()
    elif method == "min":
        grouped_champ_df = tierlist_df.groupby(by=['champion']).min()
    else:
        raise ValueError("method param must be mean, median, std, max or min")

    fig = px.line(
        grouped_champ_df,
        x=grouped_champ_df.index,
        y=information,
    )

    fig.update_layout(
        title=f"{information} grouped by {method}",
        xaxis_title="Champion",
        yaxis_title=f"{information}"
    )

    return fig

In [6]:
patch_list =[13.9,13.8,13.7,13.6,13.5,13.4,13.3,13.1]

tier_data = get_tierlist(patch_list)
fig_tierlist_grouped(tier_data)